# Trading Summary Module User Guide

This notebook provides a comprehensive guide to using the `trading_summary` module from the `algoshort` package for pre-market trading readiness analysis.

## Table of Contents

1. [Setup and Installation](#1-setup-and-installation)
2. [Understanding the Module](#2-understanding-the-module)
3. [Single Symbol Summary](#3-single-symbol-summary)
4. [Multi-Symbol Summary](#4-multi-symbol-summary)
5. [Trading Dashboard](#5-trading-dashboard)
6. [Integration with Full Pipeline](#6-integration-with-full-pipeline)
7. [Customization Options](#7-customization-options)
8. [Best Practices](#8-best-practices)

## 1. Setup and Installation

First, let's import the required modules.

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import logging
from datetime import datetime, timedelta

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Import trading summary module
from algoshort.trading_summary import (
    get_trading_summary,
    get_multi_symbol_summary,
    print_trading_summary,
    print_multi_symbol_summary,
    create_trading_dashboard,
)

print("Imports successful!")
print("\nAvailable functions:")
print("  - get_trading_summary()")
print("  - get_multi_symbol_summary()")
print("  - print_trading_summary()")
print("  - print_multi_symbol_summary()")
print("  - create_trading_dashboard()")

## 2. Understanding the Module

### Purpose

The `trading_summary` module helps you prepare for the next trading day by providing:

| Feature | Description |
|---------|-------------|
| **Current Position** | LONG, SHORT, or FLAT |
| **Trade Action** | What to do tomorrow (enter, exit, hold, flip) |
| **Stop Loss** | Current stop loss level and risk percentage |
| **Position Sizing** | Recommended share quantities by method |
| **Recent History** | Last N bars of signal history |

### Trade Actions Detected

| Action | Description |
|--------|-------------|
| `ENTER LONG` | Signal changed from 0 to 1 |
| `ENTER SHORT` | Signal changed from 0 to -1 |
| `EXIT LONG` | Signal changed from 1 to 0 |
| `EXIT SHORT` | Signal changed from -1 to 0 |
| `FLIP: LONG → SHORT` | Signal changed from 1 to -1 |
| `FLIP: SHORT → LONG` | Signal changed from -1 to 1 |
| `HOLD LONG` | Signal stayed at 1 |
| `HOLD SHORT` | Signal stayed at -1 |
| `STAY FLAT` | Signal stayed at 0 |

## 3. Single Symbol Summary

Let's create sample data and generate a trading summary for a single symbol.

In [ ]:
# Create sample data for AAPL
np.random.seed(42)
n_days = 30
dates = pd.date_range(end=datetime.now().date(), periods=n_days, freq='B')

# Simulate price movement
close = 185 + np.cumsum(np.random.randn(n_days) * 2)

# Create signal: flat -> long -> flat -> long (new entry on last day)
signal = np.zeros(n_days)
signal[5:15] = 1   # Long position
signal[15:25] = 0  # Exit
signal[25:] = 1    # Re-enter long

# Make last bar a new entry (signal change)
signal[-2] = 0
signal[-1] = 1

df_aapl = pd.DataFrame({
    'date': dates,
    'close': close,
    'hybrid_signal': signal,
    'stop_loss': close * 0.95,  # 5% trailing stop
    'shs_eql': (50000 / close).astype(int),  # Equal weight sizing
    'shs_fxd': 100,  # Fixed 100 shares
    'shs_ccv': (40000 / close).astype(int),  # Concave sizing
    'shs_cvx': (60000 / close).astype(int),  # Convex sizing
})

print(f"Sample AAPL data created:")
print(f"  Date range: {dates[0].strftime('%Y-%m-%d')} to {dates[-1].strftime('%Y-%m-%d')}")
print(f"  Current price: ${close[-1]:.2f}")
print(f"  Current signal: {int(signal[-1])}")
print(f"  Previous signal: {int(signal[-2])}")
df_aapl.tail()

In [ ]:
# Generate trading summary
summary = get_trading_summary(
    df=df_aapl,
    ticker='AAPL',
    signal_col='hybrid_signal',
    stop_loss_col='stop_loss',
    lookback=5
)

# Print formatted summary
print_trading_summary(summary, detailed=True)

In [ ]:
# Examine the raw summary dictionary
print("Raw summary dictionary keys:")
for key, value in summary.items():
    if key == 'recent_history':
        print(f"  {key}: [{len(value)} bars]")
    elif key == 'position_sizes':
        print(f"  {key}: {value}")
    else:
        print(f"  {key}: {value}")

### Example: Different Trade Actions

Let's see examples of different trade actions.

In [ ]:
# Helper function to create test data with specific signal pattern
def create_test_data(signal_pattern, ticker='TEST'):
    """Create test DataFrame with given signal pattern."""
    n = len(signal_pattern)
    return pd.DataFrame({
        'date': pd.date_range(end=datetime.now().date(), periods=n, freq='B'),
        'close': [100 + i for i in range(n)],
        'hybrid_signal': signal_pattern,
        'stop_loss': [95 + i for i in range(n)],
    })

# Test different scenarios
scenarios = [
    ('ENTER LONG', [0, 0, 0, 0, 1]),
    ('ENTER SHORT', [0, 0, 0, 0, -1]),
    ('EXIT LONG', [1, 1, 1, 1, 0]),
    ('EXIT SHORT', [-1, -1, -1, -1, 0]),
    ('FLIP L→S', [1, 1, 1, 1, -1]),
    ('FLIP S→L', [-1, -1, -1, -1, 1]),
    ('HOLD LONG', [1, 1, 1, 1, 1]),
    ('HOLD SHORT', [-1, -1, -1, -1, -1]),
    ('STAY FLAT', [0, 0, 0, 0, 0]),
]

print("Trade Action Detection Examples:")
print("=" * 60)
print(f"{'Scenario':<15} {'Signal Pattern':<20} {'Detected Action'}")
print("-" * 60)

for name, pattern in scenarios:
    df = create_test_data(pattern)
    summary = get_trading_summary(df, 'TEST')
    pattern_str = ' → '.join(map(str, pattern[-3:]))
    changed = '⚠️' if summary['signal_changed'] else '  '
    print(f"{name:<15} [{pattern_str}] {changed}{summary['trade_action']}")

## 4. Multi-Symbol Summary

For multiple symbols, use `get_multi_symbol_summary()` and `print_multi_symbol_summary()`.

In [ ]:
# Create sample data for multiple symbols
np.random.seed(123)
n_days = 30
dates = pd.date_range(end=datetime.now().date(), periods=n_days, freq='B')

# MSFT - Currently short, holding
msft_close = 420 + np.cumsum(np.random.randn(n_days) * 3)
msft_signal = np.zeros(n_days)
msft_signal[10:] = -1  # Short position

df_msft = pd.DataFrame({
    'date': dates,
    'close': msft_close,
    'hybrid_signal': msft_signal,
    'stop_loss': msft_close * 1.05,  # Stop above for shorts
    'shs_eql': (50000 / msft_close).astype(int),
    'shs_fxd': 50,
})

# GOOGL - Flat, staying flat
googl_close = 175 + np.cumsum(np.random.randn(n_days) * 2)
googl_signal = np.zeros(n_days)  # Always flat

df_googl = pd.DataFrame({
    'date': dates,
    'close': googl_close,
    'hybrid_signal': googl_signal,
    'stop_loss': googl_close * 0.95,
    'shs_eql': (50000 / googl_close).astype(int),
    'shs_fxd': 100,
})

# NVDA - Exiting long position
nvda_close = 880 + np.cumsum(np.random.randn(n_days) * 5)
nvda_signal = np.ones(n_days)
nvda_signal[-1] = 0  # Exit on last bar

df_nvda = pd.DataFrame({
    'date': dates,
    'close': nvda_close,
    'hybrid_signal': nvda_signal,
    'stop_loss': nvda_close * 0.95,
    'shs_eql': (50000 / nvda_close).astype(int),
    'shs_fxd': 25,
})

# Combine into dict
portfolio = {
    'AAPL': df_aapl,   # Enter long (from earlier)
    'MSFT': df_msft,   # Hold short
    'GOOGL': df_googl, # Stay flat
    'NVDA': df_nvda,   # Exit long
}

print(f"Portfolio created with {len(portfolio)} symbols:")
for ticker in portfolio:
    print(f"  - {ticker}")

In [ ]:
# Generate multi-symbol summary
summaries = get_multi_symbol_summary(portfolio, signal_col='hybrid_signal')

# Print the report
print_multi_symbol_summary(summaries, detailed=False)

In [ ]:
# Print with detailed view for each symbol
print_multi_symbol_summary(summaries, detailed=True)

## 5. Trading Dashboard

For programmatic access, use `create_trading_dashboard()` to get a DataFrame.

In [ ]:
# Create dashboard DataFrame
dashboard = create_trading_dashboard(portfolio, signal_col='hybrid_signal')

print("Trading Dashboard:")
dashboard

In [ ]:
# Filter for action items only (signals that changed)
action_items = dashboard[dashboard['signal_changed'] == True]

print("\n⚠️ ACTION ITEMS (signals changed):")
if len(action_items) > 0:
    print(action_items[['ticker', 'price', 'position', 'action', 'stop_loss']])
else:
    print("No action items - all positions unchanged.")

In [ ]:
# Filter by position type
long_positions = dashboard[dashboard['position'] == 'LONG']
short_positions = dashboard[dashboard['position'] == 'SHORT']
flat_positions = dashboard[dashboard['position'] == 'FLAT']

print(f"Position Summary:")
print(f"  LONG:  {len(long_positions)} positions")
print(f"  SHORT: {len(short_positions)} positions")
print(f"  FLAT:  {len(flat_positions)} positions")

if len(long_positions) > 0:
    print(f"\nLong positions: {list(long_positions['ticker'])}")
if len(short_positions) > 0:
    print(f"Short positions: {list(short_positions['ticker'])}")

## 6. Integration with Full Pipeline

Here's how to integrate the trading summary with the complete `algoshort` workflow.

In [ ]:
# Example workflow (conceptual - requires full data pipeline)
print("""
COMPLETE WORKFLOW EXAMPLE
=========================

# 1. Load data for multiple symbols
from algoshort import YFinanceDataHandler

handler = YFinanceDataHandler()
symbols = ['AAPL', 'MSFT', 'GOOGL']

portfolio = {}
for symbol in symbols:
    df = handler.get_ohlc_data(symbol, period='1y')
    portfolio[symbol] = df

# 2. Calculate relative prices
from algoshort import OHLCProcessor

for symbol, df in portfolio.items():
    processor = OHLCProcessor(df, benchmark='SPY')
    portfolio[symbol] = processor.calculate_relative_prices()

# 3. Detect regimes
from algoshort import RegimeDetector

for symbol, df in portfolio.items():
    detector = RegimeDetector(df)
    portfolio[symbol] = detector.compute('floor_ceiling')

# 4. Generate signals
from algoshort.combiner import HybridSignalCombiner

for symbol, df in portfolio.items():
    combiner = HybridSignalCombiner(
        direction_col='rrg',
        entry_col='rbo_50',
        exit_col='rtt_5010'
    )
    portfolio[symbol] = combiner.combine_signals(df, 'hybrid_signal')

# 5. Calculate stop losses
from algoshort import StopLossCalculator

for symbol, df in portfolio.items():
    calc = StopLossCalculator(df)
    df['stop_loss'] = calc.get_stop_loss('hybrid_signal', 'atr', multiplier=2.0)
    portfolio[symbol] = df

# 6. Calculate position sizes
from algoshort import PositionSizing

sizer = PositionSizing(
    tolerance=-0.1,
    mn=0.01,
    mx=0.05,
    equal_weight=0.02,
    avg=0.025,
    lot=1
)

for symbol, df in portfolio.items():
    # Calculate shares for each method
    df['shs_eql'] = (50000 / df['close']).astype(int)
    portfolio[symbol] = df

# 7. Generate trading summary
from algoshort import (
    get_multi_symbol_summary,
    print_multi_symbol_summary,
    create_trading_dashboard
)

summaries = get_multi_symbol_summary(portfolio, signal_col='hybrid_signal')
print_multi_symbol_summary(summaries)

# Get DataFrame for further analysis
dashboard = create_trading_dashboard(portfolio)
action_items = dashboard[dashboard['signal_changed']]
print(f"\nAction items for tomorrow: {len(action_items)}")
""")

## 7. Customization Options

### Custom Column Names

If your DataFrame uses different column names, specify them explicitly.

In [ ]:
# Example with custom column names
df_custom = df_aapl.rename(columns={
    'hybrid_signal': 'my_signal',
    'stop_loss': 'trailing_stop',
    'close': 'adj_close',
})

summary = get_trading_summary(
    df=df_custom,
    ticker='AAPL',
    signal_col='my_signal',
    stop_loss_col='trailing_stop',
    close_col='adj_close',
)

print(f"Custom column summary:")
print(f"  Position: {summary['position_direction']}")
print(f"  Action: {summary['trade_action']}")

In [ ]:
# Custom position sizing columns
df_custom_pos = df_aapl.copy()
df_custom_pos['risk_parity_shares'] = 75
df_custom_pos['kelly_shares'] = 120

summary = get_trading_summary(
    df=df_custom_pos,
    ticker='AAPL',
    position_cols={
        'Risk Parity': 'risk_parity_shares',
        'Kelly': 'kelly_shares',
    }
)

print(f"Position sizes with custom methods:")
for method, shares in summary['position_sizes'].items():
    print(f"  {method}: {shares} shares")

### Adjusting Lookback Period

In [ ]:
# Short lookback (3 bars)
summary_short = get_trading_summary(df_aapl, 'AAPL', lookback=3)
print(f"Short lookback (3 bars): {len(summary_short['recent_history'])} history entries")

# Longer lookback (10 bars)
summary_long = get_trading_summary(df_aapl, 'AAPL', lookback=10)
print(f"Long lookback (10 bars): {len(summary_long['recent_history'])} history entries")

## 8. Best Practices

### Required Columns

| Column | Required | Default Name | Description |
|--------|----------|--------------|-------------|
| Date | Yes | `date` | Date/datetime column or index |
| Close | Yes | `close` | Closing price |
| Signal | Yes | `hybrid_signal` | Signal column (1, -1, 0) |
| Stop Loss | No | `stop_loss` | Stop loss price |
| Position Sizes | No | `shs_*` | Share quantities |

### Position Sizing Column Names

The module auto-detects these column names:
- `shs_eql` → equal_weight
- `shs_fxd` → fixed
- `shs_ccv` → concave
- `shs_cvx` → convex

In [ ]:
# Quick pre-market routine
def pre_market_check(portfolio, signal_col='hybrid_signal'):
    """
    Quick pre-market routine to check all positions.
    
    Returns action items that need attention.
    """
    # Get summaries
    summaries = get_multi_symbol_summary(portfolio, signal_col=signal_col)
    
    # Print report
    print_multi_symbol_summary(summaries)
    
    # Return action items
    dashboard = create_trading_dashboard(portfolio, signal_col=signal_col)
    actions = dashboard[dashboard['signal_changed'] == True]
    
    return actions

# Run pre-market check
actions = pre_market_check(portfolio)

if len(actions) > 0:
    print("\n📋 Export action items:")
    print(actions[['ticker', 'action', 'price', 'stop_loss']].to_string(index=False))

In [ ]:
# Quick reference card
print("""
TRADING SUMMARY MODULE - QUICK REFERENCE
========================================

# Import
from algoshort import (
    get_trading_summary,
    get_multi_symbol_summary,
    print_trading_summary,
    print_multi_symbol_summary,
    create_trading_dashboard,
)

# Single symbol
summary = get_trading_summary(df, 'AAPL', signal_col='hybrid_signal')
print_trading_summary(summary)

# Multiple symbols
portfolio = {'AAPL': df_aapl, 'MSFT': df_msft}
summaries = get_multi_symbol_summary(portfolio)
print_multi_symbol_summary(summaries)

# Dashboard DataFrame
dashboard = create_trading_dashboard(portfolio)
action_items = dashboard[dashboard['signal_changed']]

# Key fields in summary:
#   summary['position_direction']  # LONG, SHORT, FLAT
#   summary['trade_action']        # ENTER LONG, EXIT SHORT, etc.
#   summary['signal_changed']      # True if action needed
#   summary['stop_loss']           # Current stop level
#   summary['risk_pct']            # Risk percentage
#   summary['position_sizes']      # Dict of shares by method
""")

---

## Summary

This guide covered:

1. **Setup**: Import and understand the trading summary module
2. **Module Overview**: Trade actions detected and key features
3. **Single Symbol**: Generate and print summary for one ticker
4. **Multi-Symbol**: Batch processing for portfolios
5. **Dashboard**: DataFrame output for programmatic access
6. **Integration**: Full pipeline workflow example
7. **Customization**: Custom column names and lookback periods
8. **Best Practices**: Required columns and pre-market routine

### Key Takeaways

- Use `get_trading_summary()` for single symbol analysis
- Use `get_multi_symbol_summary()` for portfolio-wide view
- Use `create_trading_dashboard()` for DataFrame output
- Check `signal_changed` to identify action items
- Run this as part of your daily pre-market routine

For questions or issues, refer to the test suite at `tests/test_trading_summary.py`.